# Part 1 - Optical Flow

- Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera.
- **Optical Flow Analysis has a few assumptions:**
    - The pixel intensities of an object do not change between consecutive frames.
    - neighbouring pixels have similar motion.
- The optical flow methods in OpenCV will first take in a given set of points and a frame.
- Then it will attempt to find those points in the next frame.
- It is up to the user to supply the points to track.

![Example Optical Flow](https://docs.opencv.org/3.4/optical_flow_basic1.jpg)

- Note that given just this clip, we can not determine if the ball is moving, or if the camera moved down and to the left!
- Using OpenCV we pass in the previous frame, previous points and the current frame to the **Lucas-kanade** function.
- The Lucas-kanade computes optical flow for a sparse feature set.
    - Meaning only the points it was told to track.
- But what if we wanted to track all the points in a video?
- We can use **Gunner Farneback's algorithm** (also built in to OpenCV) to calculate **dense** optical flow.
- This **dense** optical flow will calculate flow for all points in an image.
- It will color them black if no flow (no movement) is detected.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [5]:
### Optical Flow - Lucas Kanade method ###

corner_track_params = dict(maxCorners = 10, qualityLevel=0.3, minDistance=7, blockSize=7)

#  Lucas-kanade parameters
lk_params = dict(winSize=(200, 200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv2.VideoCapture(0)

# read the very first frame
ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Optical tracking
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(
        prevImg = prev_gray,
        nextImg = frame_gray,
        prevPts = prevPts,
        nextPts = None, 
        **lk_params
    )
    
    good_new  = nextPts[status == 1]
    good_prev = nextPts[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        
        # ravel only flattens the numpy array
        x_new, y_new   = new.ravel()
        x_new, y_new = int(x_new), int(y_new)
        x_prev, y_prev = prev.ravel()
        x_prev, y_prev = int(x_prev), int(y_prev)
        
        mask = cv2.line(mask, pt1=(x_new, y_new), pt2=(x_prev, y_prev), color=(0, 255, 0), thickness=3)
        
        frame = cv2.circle(frame, center=(x_new, y_new), radius=8, color=(0, 0, 255), thickness=-1)
        
    img = cv2.add(frame, mask)
    cv2.imshow("tracking", img)
    
    if cv2.waitKey(30) & 0xFF == ord("q"):
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
        
cap.release()
cv2.destroyAllWindows()

In [6]:
### Optical flow - Dense Optical Flow ###

cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prevImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:, :, 1] = 255

while True:
    
    ret, frame2 = cap.read()
    
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # this flow gives the cartesian direction
    flow = cv2.calcOpticalFlowFarneback(
        prev = prevImg,
        next = nextImg,
        flow = None,
        pyr_scale = 0.5, 
        levels = 3,
        winsize = 15,
        iterations = 3,
        poly_n = 5,
        poly_sigma = 1.2,
        flags = 0
    )
    
    # by-default, it returns radian, get angle by passing the last parameter
    mag, ang = cv2.cartToPolar(x=flow[:, :, 0], y=flow[:, :, 1], angleInDegrees=True)
    
    hsv_mask[:, :, 0] = ang / 2
    hsv_mask[:, :, 2] = cv2.normalize(mag, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow("frame", bgr)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
        
    prevImg = nextImg
    
cap.release()
cv2.destroyAllWindows()